In [ ]:
#default_exp rxtransform

In [ ]:
#export
from fastai2.basics import *
from lazylabel.core import *

# Reactive Transforms
> Adds the ability to listen on transforms.

In [ ]:
#export
def _maintain_labels(old, new):
    if isinstance(new, tuple): return new
    return add_attr(new, 'labels', getattr(old, 'labels', []))

In [ ]:
#export
def _hash_fn(o): return o.__name__

## Makes transforms listenables

In [ ]:
#export
@patch
def __new__(cls:Transform, *args, **kwargs):
    obj = super(Transform, cls).__new__(cls)
    obj.listeners = OrderedDict()
    obj.pre_broad = dict()
    return obj

In [ ]:
#export
_old_do_call = Transform._do_call
@patch
def _do_call(self:Transform, f, x, **kwargs):
    res = _maintain_labels(x, _old_do_call(self, f, x, **kwargs))
    if f[type(x)] is not None: # Checks for type dispatch
        self._pres = {k: v(res) for k,v in self.pre_broad.items()}
        self.broadcast(res)
    return res

In [ ]:
#export
@patch
def broadcast(self:Transform, x):
    for f,pre in self.listeners.values(): f(x, *[self._pres[_hash_fn(o)] for o in pre])

In [ ]:
#export
class Subscription:
    def __init__(self, f, cancel):
        self.f,self.cancel,self.sub = f,cancel,None
        
    @property
    def __name__(self): return self.f.__name__
        
    def __call__(self, *args, **kwargs):
        if self.sub is not None: self.sub(self.__name__)
        self.sub = None
        return self.f(*args, **kwargs)
    
    def listen_one(self, f): self.sub = f
    
    def cancel(self):
        self.cancel()
        self.sub = None

In [ ]:
#export
@patch
def listen(self:Transform, *pre):
    for o in pre: self.pre_broad[_hash_fn(o)] = o
    def _inner(f):
        f = Subscription(f, lambda: self.listeners.pop(f_hash))
        f_hash = _hash_fn(f)
        self.listeners[f_hash] = (f, pre)
        return f
    return _inner

In [ ]:
@Transform
def tfm(x:float): return -x

In [ ]:
def IntPre(x):
    print('IntPre') # Should only print once per transform call
    return int(x)
def StrPre(x): return str(x)

In [ ]:
res = []
@tfm.listen()
def f0(x): res.append(x)
@tfm.listen(IntPre, StrPre)
def f1(x, v1, v2): res.append([x, v1, v2])
@tfm.listen(StrPre, IntPre)
def f2(x, v1, v2): res.append([x, v1, v2])
@tfm.listen(IntPre)
def f3(x, v1): res.append([x, v1])
@tfm.listen(IntPre)
def f3(x, v1): res.append([x, v1])

In [ ]:
res.clear()
tfm(4.2)
test_eq(res, [-4.2, [-4.2, -4, '-4.2'], [-4.2, '-4.2', -4], [-4.2, -4]])

IntPre


In [ ]:
res.clear()
tfm(4)
test_eq(res, [])

In [ ]:
f1.cancel()
res.clear()
tfm(4.2)
test_eq(res, [-4.2, [-4.2, '-4.2', -4], [-4.2, -4]])

IntPre


In [ ]:
order = []
for f in [f0,f1,f2,f3]: f.listen_one(order.append)
tfm(4.2)
test_eq(order, ['f0', 'f2', 'f3'])

IntPre


# Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_rxtransform.ipynb.
Converted 02_labeller.core.ipynb.
Converted 02a_labeller.metrics.ipynb.
Converted 03_model.majority_label_voter.ipynb.
Converted 05_text.core.ipynb.
Converted 06_text.labellers.ipynb.
Converted index.ipynb.
